<img src="SQLite_logo.png" width="500" height="300" align="right"/>

# SQL: Read Tables From DataBase

In [1]:
import pandas as pd
import sqlite3 as sql

In [2]:
to_server = r'.\My_Customer_Warehouse.db'
conn = sql.connect(to_server)
c = conn.cursor()

## Get Info (header) about all tables in the DataBase 

In [3]:
def tables_in_sqlite_db(conn):
    cursor = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = [
        v[0] for v in cursor.fetchall()
        if v[0] != "sqlite_sequence"
    ]
    cursor.close()
    return tables

In [4]:
tables = tables_in_sqlite_db(conn)
sql.collections.ChainMap(tables)

ChainMap(['agents', 'company', 'customer', 'orders', 'daysorder', 'despatch', 'foods', 'listofitem', 'student', 'studentreport', 'customer_analytics', 'Income_Expense_Data', 'online_shoppers_intention'])

## Get additional info about a table

In [5]:
c.execute("PRAGMA table_info(orders)")
c.fetchall()

[(0, 'ORD_NUM', 'numeric', 0, None, 0),
 (1, 'ORD_AMOUNT', 'numeric', 0, None, 0),
 (2, 'ADVANCE_AMOUNT', 'numeric', 0, None, 0),
 (3, 'ORD_DATE', 'date', 0, None, 0),
 (4, 'CUST_CODE', 'numeric', 0, None, 0),
 (5, 'AGENT_CODE', 'text', 0, None, 0),
 (6, 'ORD_DESCRIPTION', 'text', 0, None, 0)]

## See all Rows From a Table

In [6]:
conn.row_factory = sql.Row
cursor = conn.execute('select * from customer')
# instead of cursor.description:
row = cursor.fetchone()
names = row.keys()
names




['CUST_CODE',
 'CUST_NAME',
 'CUST_CITY',
 'WORKING_AREA',
 'CUST_COUNTRY',
 'GRADE',
 'OPENING_AMT',
 'RECEIVE_AMT',
 'PAYMENT_AMT',
 'OUTSTANDING_AMT',
 'PHONE_NO',
 'AGENT_CODE']

## SQL Queries: Examples

In [7]:
select = ['select * from orders limit 3',
        "SELECT * FROM customer WHERE customer.CUST_NAME LIKE 'M%'",
        "SELECT * FROM customer WHERE CUST_NAME LIKE '%a'",
        'Select CUST_CODE, WORKING_AREA from customer',
         "SELECT distinct CUST_CODE FROM customer"]

select_join=['Select * from customer inner join orders',
        "Select * from customer inner join orders where customer.CUST_CODE = orders.CUST_CODE and orders.CUST_CODE = 'C00015'"]

select_where=["SELECT CUST_NAME FROM customer where CUST_NAME between 'Charles' and 'Michael'"]

select_groupby=['Select * from orders group by CUST_CODE order by ORD_AMOUNT']


## Execute Queries using Pandas

### getting table info

In [8]:
pd.read_sql_query("""
PRAGMA table_info(customer);
""", conn)

,cid,name,type,notnull,dflt_value,pk
0,0,CUST_CODE,text,0,None,0
1,1,CUST_NAME,text,0,None,0
2,2,CUST_CITY,text,0,None,0
3,3,WORKING_AREA,text,0,None,0
4,4,CUST_COUNTRY,text,0,None,0
5,5,GRADE,numeric,0,None,0
6,6,OPENING_AMT,numeric,0,None,0
7,7,RECEIVE_AMT,numeric,0,None,0
8,8,PAYMENT_AMT,numeric,0,None,0
9,9,OUTSTANDING_AMT,numeric,0,None,0


In [9]:
result = pd.read_sql(select[0], conn)
result.head()
    

,ORD_NUM,ORD_AMOUNT,ADVANCE_AMOUNT,ORD_DATE,CUST_CODE,AGENT_CODE,ORD_DESCRIPTION
0,200100,1000,600,2008-01-08,C00015,A003,SOD\r
1,200110,3000,500,2008-04-15,C00019,A010,SOD\r
2,200107,4500,900,2008-08-30,C00007,A010,SOD\r


In [10]:
result = pd.read_sql(select[1], conn)
result.head()

,CUST_CODE,CUST_NAME,CUST_CITY,WORKING_AREA,CUST_COUNTRY,GRADE,OPENING_AMT,RECEIVE_AMT,PAYMENT_AMT,OUTSTANDING_AMT,PHONE_NO,AGENT_CODE
0,C00001,Micheal,New York,New York,USA,2,3000,5000,2000,6000,CCCCCCC,A008
1,C00003,Martin,Torento,Torento,Canada,2,8000,7000,7000,8000,MJYURFD,A004


In [11]:
result = pd.read_sql(select[2], conn)
result.head()

,CUST_CODE,CUST_NAME,CUST_CITY,WORKING_AREA,CUST_COUNTRY,GRADE,OPENING_AMT,RECEIVE_AMT,PAYMENT_AMT,OUTSTANDING_AMT,PHONE_NO,AGENT_CODE
0,C00008,Karolina,Torento,Torento,Canada,1,7000,7000,9000,5000,HJKORED,A004
1,C00014,Rangarappa,Bangalore,Bangalore,India,2,8000,11000,7000,12000,AAAATGF,A001
2,C00011,Sundariya,Chennai,Chennai,India,3,7000,11000,7000,11000,PPHGRTS,A010


In [12]:
result = pd.read_sql(select[3], conn)
result.head()

,CUST_CODE,WORKING_AREA
0,C00013,London
1,C00001,New York
2,C00020,New York
3,C00025,Bangalore
4,C00024,London


In [13]:
result = pd.read_sql(select[4], conn)
result.head()

,CUST_CODE
0,C00013
1,C00001
2,C00020
3,C00025
4,C00024


In [14]:
result = pd.read_sql(select_join[0], conn)
result.head()

,CUST_CODE,CUST_NAME,CUST_CITY,WORKING_AREA,CUST_COUNTRY,GRADE,OPENING_AMT,RECEIVE_AMT,PAYMENT_AMT,OUTSTANDING_AMT,PHONE_NO,AGENT_CODE,ORD_NUM,ORD_AMOUNT,ADVANCE_AMOUNT,ORD_DATE,CUST_CODE,AGENT_CODE,ORD_DESCRIPTION
0,C00013,Holmes,London,London,UK,2,6000,5000,7000,4000,BBBBBBB,A003,200100,1000,600,2008-01-08,C00015,A003,SOD\r
1,C00013,Holmes,London,London,UK,2,6000,5000,7000,4000,BBBBBBB,A003,200110,3000,500,2008-04-15,C00019,A010,SOD\r
2,C00013,Holmes,London,London,UK,2,6000,5000,7000,4000,BBBBBBB,A003,200107,4500,900,2008-08-30,C00007,A010,SOD\r
3,C00013,Holmes,London,London,UK,2,6000,5000,7000,4000,BBBBBBB,A003,200112,2000,400,2008-05-30,C00016,A007,SOD\r
4,C00013,Holmes,London,London,UK,2,6000,5000,7000,4000,BBBBBBB,A003,200113,4000,600,2008-06-10,C00022,A002,SOD\r


In [15]:
result = pd.read_sql(select_join[1], conn)
result.head()

,CUST_CODE,CUST_NAME,CUST_CITY,WORKING_AREA,CUST_COUNTRY,GRADE,OPENING_AMT,RECEIVE_AMT,PAYMENT_AMT,OUTSTANDING_AMT,PHONE_NO,AGENT_CODE,ORD_NUM,ORD_AMOUNT,ADVANCE_AMOUNT,ORD_DATE,CUST_CODE,AGENT_CODE,ORD_DESCRIPTION
0,C00015,Stuart,London,London,UK,1,6000,8000,3000,11000,GFSGERS,A003,200100,1000,600,2008-01-08,C00015,A003,SOD\r
1,C00015,Stuart,London,London,UK,1,6000,8000,3000,11000,GFSGERS,A003,200127,2500,400,2008-07-20,C00015,A003,SOD\r


In [16]:
result = pd.read_sql(select_where[0], conn)
result.head()

,CUST_NAME
0,Holmes
1,Cook
2,Fleming
3,Jacks
4,Karl


In [17]:
result = pd.read_sql(select_groupby[0], conn)
result.head()

,ORD_NUM,ORD_AMOUNT,ADVANCE_AMOUNT,ORD_DATE,CUST_CODE,AGENT_CODE,ORD_DESCRIPTION
0,200120,500,100,2008-07-20,C00009,A002,SOD\r
1,200116,500,100,2008-07-13,C00010,A009,SOD\r
2,200124,500,100,2008-06-20,C00017,A007,SOD\r
3,200118,500,100,2008-07-20,C00023,A006,SOD\r
4,200117,800,200,2008-10-20,C00014,A001,SOD\r


In [18]:
conn.commit()
conn.close()